In [150]:
%load_ext IPython.extensions.autoreload
%autoreload 2

The IPython.extensions.autoreload extension is already loaded. To reload it, use:
  %reload_ext IPython.extensions.autoreload


In [151]:
import sys
sys.path.append('../..')
from model import FinData
from model import merged_split
from model import CatboostFinModel
from model import train_valid_test_split
import datetime as dt

In [152]:
data = FinData("../../datasets/Sber_10_min.csv")

windows_shifts_norms = [2, 3, 4, 5, 6]
windows_ma = [2, 4, 6, 8, 18, 30, 50]
windows_ema = [3, 5, 7, 9, 20, 35, 100]

data.insert_shifts_norms(windows_shifts_norms=windows_shifts_norms)
data.insert_rolling_means(windows_ma=windows_ma)
data.insert_exp_rolling_means(windows_ema=windows_ema)
data.insert_bollinger()
data.insert_rsi()
data.insert_high_low_diff()
data.insert_stochastic_oscillator()
data.insert_random_prediction()

In [153]:
data.make_long_strat_target(target_name="is_long", commission=0.0004)

In [154]:
num = data.get_numeric_features()
cat = data.get_cat_features()
target = "is_long"

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_valid_test_split(data.df, 
                                                                        start_period=dt.datetime(2024, 12, 1), 
                                                                        train_size=3000, 
                                                                        val_size=500, 
                                                                        test_size=500, 
                                                                        numeric=num, cat=cat, target=target, silenced=False)

Начало тренировочного периода: 2024-11-01 04:00:00. Конец тренировочного периода: 2024-12-01 06:30:00 
                     Начало валидационного периода: 2024-12-01 06:40:00. Конец валидационного периода: 2024-12-06 07:10:00 
                     Начало тестового периода: 2024-12-06 07:20:00. Конец тестового периода: 2024-12-11 07:20:00 
 


In [170]:
args = {"iterations" : 3000, 
        "depth" : 4, 
        "learning_rate" : 0.01, 
        "use_best_model" : True, 
        "l2_leaf_reg" : 200,
        "scale_pos_weight" : 0.9,
        "loss_function" : 'Logloss', 
        "eval_metric" : 'Logloss', 
        "cat_features" : cat, 
        "random_state" : 42,
        "early_stopping_rounds" : 1000}

In [171]:
model = CatboostFinModel(args = args)

model.set_datasets(X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val)
model.set_features(numeric_features=num, cat_features=cat)

model.fit()

0:	learn: 0.6903354	test: 0.6906146	best: 0.6906146 (0)	total: 5.63ms	remaining: 16.9s
1:	learn: 0.6878784	test: 0.6883565	best: 0.6883565 (1)	total: 10.7ms	remaining: 16s
2:	learn: 0.6854655	test: 0.6861668	best: 0.6861668 (2)	total: 18ms	remaining: 17.9s
3:	learn: 0.6829594	test: 0.6838900	best: 0.6838900 (3)	total: 23.2ms	remaining: 17.4s
4:	learn: 0.6804826	test: 0.6816200	best: 0.6816200 (4)	total: 29.7ms	remaining: 17.8s
5:	learn: 0.6781061	test: 0.6795162	best: 0.6795162 (5)	total: 36.2ms	remaining: 18.1s
6:	learn: 0.6755606	test: 0.6773578	best: 0.6773578 (6)	total: 44.3ms	remaining: 18.9s
7:	learn: 0.6730532	test: 0.6749896	best: 0.6749896 (7)	total: 54.1ms	remaining: 20.2s
8:	learn: 0.6707469	test: 0.6728671	best: 0.6728671 (8)	total: 60.7ms	remaining: 20.2s
9:	learn: 0.6684232	test: 0.6707212	best: 0.6707212 (9)	total: 69.3ms	remaining: 20.7s
10:	learn: 0.6660137	test: 0.6684733	best: 0.6684733 (10)	total: 75.1ms	remaining: 20.4s
11:	learn: 0.6640773	test: 0.6668719	best: 0.

In [172]:
print(model.score(X_test, y_test))

              precision    recall  f1-score   support

           0       0.73      0.96      0.83       361
           1       0.50      0.09      0.16       139

    accuracy                           0.72       500
   macro avg       0.62      0.53      0.50       500
weighted avg       0.67      0.72      0.65       500



In [173]:
print(model.score(X_val, y_val))

              precision    recall  f1-score   support

           0       0.71      0.97      0.82       345
           1       0.67      0.12      0.20       155

    accuracy                           0.71       500
   macro avg       0.69      0.55      0.51       500
weighted avg       0.70      0.71      0.63       500



In [174]:
print(model.get_constant_accuracy(y_test, target_name="is_long"))

0.722
